In [1]:

from notebook_functions import *

############# PARAMETERS #############

parameters = dict()
parameters['dynamic_range'] = 40

######################################


## FUNCTIONS ##

def load_data(file):
    
    with closing(sql.connect(file)) as con:
        
        q = 'SELECT angle, x, y, z, brightness FROM image ORDER BY angle, x, y, z'
        table = pd.read_sql(q, con)
        
    angle = np.array(table['angle'])
    field_pos = np.array(table[['x', 'y', 'z']])
    brightness = np.array(table['brightness'])
    
    x, y, z = field_pos.T
    nangle = len(np.unique(angle))
    nx = len(np.unique(x))
    ny = len(np.unique(y))
    nz = len(np.unique(z))
    
    field_pos = field_pos.reshape((nangle, nx, ny, nz, -1))
    brightness = brightness.reshape((nangle, nx, ny, nz))
    angles = np.unique(angle)
    
    return brightness, angles, field_pos

## SCRIPT ##

path_to_file = open_file_dialog()
parameters['file'] = path_to_file

brightness, angles, field_pos = load_data(path_to_file)
x, y, z = field_pos.T
dynamic_range = parameters['dynamic_range']



In [3]:
## DISPLAY ##

display(Markdown('### <center> Notebook parameters </center>'))
display(HTML('<center>' + tabulate.tabulate(parameters.items(), tablefmt='html') + '</center>'))

@widgets.interact(a=widgets.FloatSlider(min=np.min(angles), max=np.max(angles), step=np.diff(angles)[0], description='Angle'))
def plot(a):
    
    idx = np.argmin(np.abs(a - angles))
    
    fig = plt.figure(figsize=(15, 9))
    gs = GridSpec(1, 2)
    ax1 = fig.add_subplot(gs[0, 0])
    ax2 = fig.add_subplot(gs[0, 1])
    
    extents = (np.min(x), np.max(x)), (np.min(y), np.max(y)), (np.min(z), np.max(z))
    shapes = brightness.shape[1:4]
    extent = list()
    dims = list()
    for i in range(3):
        if shapes[i] != 1:
            extent += extents[i]
            dims.append(('x', 'y', 'z')[i])
    
    bright1 = 20 * np.log10(brightness[idx, ...].squeeze().T / np.max(brightness[idx, ...]))
    bright1[bright1 < -dynamic_range] = -dynamic_range
    bright2 = 20 * np.log10(brightness[0, ...].squeeze().T / np.max(brightness[0, ...]))
    bright2[bright1 < -dynamic_range] = -dynamic_range
    
    im1 = ax1.imshow(bright1, origin='lower', extent=extent, vmin=-dynamic_range, vmax=0, interpolation='bilinear')
    im2 = ax2.imshow(bright2, origin='lower', extent=extent, vmin=-dynamic_range, vmax=0, interpolation='bilinear')
    
    add_colorbar(im1)
    add_colorbar(im2)
#     fig.colorbar(im1, ax=ax1)
#     fig.colorbar(im2, ax=ax2)
    
    ax1.set_xlabel(dims[0] + ' (m)')
    ax1.set_ylabel(dims[1] + ' (m)')
    
    ax2.set_xlabel(dims[0] + ' (m)')
    ax2.set_ylabel(dims[1] + ' (m)')
    
    plt.tight_layout()


### <center> Notebook parameters </center>

dynamic_range,40
file,D:/Projects/interaction3/folding_error_xz.db


interactive(children=(FloatSlider(value=0.0, description='Angle', max=5.0, step=1.0), Output()), _dom_classes=('widget-interact',))